# A practical introduction to metaprogramming in Julia

**Andy Ferris**, *JuliaCon 2018*

---

Welcome to the JuliaCon 2018 workshop on metaprogramming!

This is an Jupyter notebook. It contains a mixture of static text and interactive code. The Jupyter notebook is connected to  a fully-fledged instance of Julia (the "kernel"), so you can execute any code like you would at the REPL.

In [1]:
## EXERCISE

# Use shift-enter to evaluate cell

1 + 1

2

In [2]:
# Feel free to type and execute whatever you want



Execute the code blocks as you follow along with the tutorial below. There will be exercises.

---

# Introduction

In many programming environments, user friendliness must be traded of with execution speed. Using abstractions or “generic” code may come with a run-time overhead. Avoiding abstract or generic code exposes the user to underlying details that they may not care about, making code harder to read and reason about at a higher level, reducing programmer productivity. Often two languages are even employed - one for rapid prototyping, and one for deployment.

It doesn’t have to be this way. The Julia language has been carefully constructed to allow for many common abstractions to be dealt with statically at compile time and have a have a zero run-time cost. This workshop will cover the topic of “metaprogramming” in Julia, which plays a large role in providing for low-cost abstractions and generic APIs. Traditionally, a “meta” program is logic which executes at compile time to help generate the code of a resulting program - that is, it is code that generates other code.

In this workshop we will cover the building blocks of metaprogramming in Julia, starting with one of its core concepts - multiple dispatch, which in combination with the type system is itself is a Turing-complete computational environment. We will then begin working our way to more advanced topics such as traits, macros, constant propagation and generated functions, following approximately this order:

* Multiple dispatch as a metaprogramming technique
* Method inlining: faster than C?
* Games with tuples: splatting, slurping and recursion
* Dispatch revisited: interfaces and traits
* Constant propagation (and what are `@pure` functions?)
* Expressions and macros
* Generated functions and when (not) to use them

This workshop will attempt to be a pedagogical tutorial on how and when to use these techniques, full of practical examples I’ve seen in the wild or have used in my own code. Consideration will be given in how to use metaprogramming and still maintain a readable code base. Advice will also be provided on how to work with the compiler, and not against it, and how to make effective use of tools such as `@code_typed`. At the end of the workshop I hope you will have learned a technique or two that will help you to create generic, user-friendly APIs without sacrificing peak performance.

In [ ]:
## QUESTIONS?

---

# How Julia works

To understand how and when to apply metaprogramming techniques, it's important to gain an understanding of how the compiler works.

Compilation in Julia can be broken down into a series of steps. Very roughly, the text you type at the REPL is turned into machine code through the following processes:

 1. **Parsing:** Text like `f(x)` is converted from a simple stream of characters to *expressions*. Expressions are of type `Expr` and are a structured and normalized representation of the code, for example `Expr(:call, :f, :x)`. These surface-level expressions are a form of "intermediate representaiton" or IR of your code. Julia has multiple levels of IR as your code is processed from our high-level language to low-level machine code.
 
 2. **Lowering:** The surface-level IR is transformed into so-called "lowered" IR through a series of syntactical transformations. These transformations mostly represent Julia's "syntactic sugar". For example, the expression for `a[i]` is transformed into the expression for `getindex(a, i)`. Broadcast fusion `f.(a .+ b)` as well as creation of closures are other examples of Julia's syntax "sugar".

During the lowering stage, the compiler also populates the type tree and generic method tables. Afterwards, your program is semantically complete and "ready to run", and can be executed by an interpretter, such as *ASTInterpreter.jl*. This is the stage that "precompilation" ends and forms the contents of `.ji` files.

However, by default Julia will perform many additional optimization steps to transform your code into fast machine code. The steps below should be viewed as optimizations and theoretically shouldn't *semantically* affect your program in any way.

 3. **Inference and optimization:** At this point the IR is in a standardized form, and we can perform analysis and transformations on the code to make it faster. Chief amongst these optimizations is type inference and method resolution - the type of each expression is *inferred* by the compiler. When a function is called, inference will determine which method of that function is dispatched to, given the input types. It must then analyze that function to determine the return type. Constant propagation, branch pruning, inlining, and other optimizations also occur at this stage.
 
 4. **LLVM code generation:** The resulting optimized, lowered Julia IR is then transformed to LLVM IR, which is a type of assembly language. LLVM will perform it's own set of optimizations on the assembly code.
 
 5. **Native code generation:** LLVM will transform this into native machine code, placing each specialized method into memory that can be called (from any language) via a function pointer and following the C calling convention.

Importantly, Julia provides powerful reflection tools to "see" what is happening at any of these stages!

In [3]:
# The surface-level expression for `1 + 1`. 
# Each `Expr` contains one "head" and many "args" (arguments). 
# For the case of `:call`, the first argument is the function being called
dump(:(1 + 1))

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Int64 1
    3: Int64 1


In [4]:
# This is looking *inside* the + function, for `+(::Int64, ::Int64)`.
# Internally, this calls another (built-in) function called `Base.add_int`
@code_lowered 1 + 1

CodeInfo(
1 ─ %1 = (Base.add_int)(x, y)
└──      return %1
)

In [5]:
# Inference has marked the return type of `+(::Int64, ::Int64)` as `Int64`.
# Base.add_int is a built-in function and cannot be optimized further.
@code_typed 1 + 1

CodeInfo(
1 ─ %1 = (Base.add_int)(x, y)::Int64
└──      return %1
) => Int64

In [6]:
# Here is the LLVM assembly for adding two `i64`s, using LLVM's built-in `add` function.
@code_llvm 1 + 1


;  @ int.jl:53 within `+'
define i64 @"julia_+_11601"(i64, i64) {
top:
  %2 = add i64 %1, %0
  ret i64 %2
}


In [7]:
# This is the native x64 assembly for adding two signed 64-bit integers, using the `leaq` instruction.
@code_native 1 + 1

	.text
; ┌ @ int.jl:53 within `+'
	leaq	(%rdi,%rsi), %rax
	retq
	nopw	%cs:(%rax,%rax)
; └


In [11]:
## EXERCISE

# Feel free to explore code for some other functions you know. Press shift-enter to execute.
# Some ideas: ==, <, push!, sin (with Int or Float64)

dump(:(sin(2)))

Expr
  head: Symbol call
  args: Array{Any}((2,))
    1: Symbol sin
    2: Int64 2


In [12]:
@code_lowered sin(2)

CodeInfo(
1 ─ %1 = (Base.Math.float)(x)
│   %2 = (Base.Math.sin)(%1)
└──      return %2
)

In [13]:
@code_typed sin(2)

CodeInfo(
1 ─ %1 = (Base.sitofp)(Float64, x)::Float64
│   %2 = invoke Base.Math.sin(%1::Float64)::Float64
└──      return %2
) => Float64

In [14]:
@code_llvm sin(2)


;  @ special/trig.jl:53 within `sin'
define double @julia_sin_12819(i64) {
top:
; ┌ @ float.jl:271 within `float'
; │┌ @ float.jl:256 within `Type' @ float.jl:60
    %1 = sitofp i64 %0 to double
; └└
  %2 = call double @julia_sin_12820(double %1)
  ret double %2
}


In [15]:
@code_native sin(2)

	.text
; ┌ @ trig.jl:53 within `sin'
; │┌ @ float.jl:271 within `float'
; ││┌ @ float.jl:256 within `Type' @ trig.jl:53
	pushq	%rax
	vcvtsi2sdq	%rdi, %xmm0, %xmm0
; │└└
	movabsq	$julia_sin_12820, %rax
	callq	*%rax
	popq	%rax
	retq
	nopw	%cs:(%rax,%rax)
; └


---

# Multiple dispatch as a metaprogramming technique

As we saw, Julia's inference engine will resolve method dispatch at compile-time. This gives us an opportunity to perform logic.

Here we will first demonstrate that Julia's dispatch system is in-fact Turing complete. Here we are building our own binary machine, which we can do computations with at compile time.

In [18]:
?:

search: : >: <:



```
(:)(start, [step], stop)
```

Range operator. `a:b` constructs a range from `a` to `b` with a step size of 1 (a [`UnitRange`](@ref)) , and `a:s:b` is similar but uses a step size of `s` (a [`StepRange`](@ref)).

`:` is also used in indexing to select whole dimensions.

---

```
(:)(I::CartesianIndex, J::CartesianIndex)
```

Construct [`CartesianIndices`](@ref) from two `CartesianIndex`.

!!! compat "Julia 1.1"
    This method requires at least Julia 1.1.


# Examples

```jldoctest
julia> I = CartesianIndex(2,1);

julia> J = CartesianIndex(3,3);

julia> I:J
2×3 CartesianIndices{2,Tuple{UnitRange{Int64},UnitRange{Int64}}}:
 CartesianIndex(2, 1)  CartesianIndex(2, 2)  CartesianIndex(2, 3)
 CartesianIndex(3, 1)  CartesianIndex(3, 2)  CartesianIndex(3, 3)
```


In [19]:
# Binary representation at the type level - no run-time data!

abstract type Bit; end

struct Zero <: Bit; end
struct One <: Bit; end

In [20]:
# OR and AND for two Bits

Base.:|(::Zero, ::Zero) = Zero()
Base.:|(::Zero, ::One)  = One()
Base.:|(::One,  ::Zero) = One()
Base.:|(::One,  ::One)  = One()

Base.:&(::Zero, ::Zero) = Zero()
Base.:&(::Zero, ::One)  = Zero()
Base.:&(::One,  ::Zero) = Zero()
Base.:&(::One,  ::One)  = One()

In [21]:
# 8 Bits make a Byte

struct Byte{Bit1 <: Bit, Bit2 <: Bit, Bit3 <: Bit, Bit4 <: Bit, Bit5 <: Bit, Bit6 <: Bit, Bit7 <: Bit, Bit8 <: Bit}
    bit1::Bit1
    bit2::Bit2
    bit3::Bit3
    bit4::Bit4
    bit5::Bit5
    bit6::Bit6
    bit7::Bit7
    bit8::Bit8
end

In [22]:
# Bitwise OR and AND on Bytes

function Base.:|(byte1::Byte, byte2::Byte)
    return Byte(byte1.bit1 | byte2.bit1,
                byte1.bit2 | byte2.bit2,
                byte1.bit3 | byte2.bit3,
                byte1.bit4 | byte2.bit4,
                byte1.bit5 | byte2.bit5,
                byte1.bit6 | byte2.bit6,
                byte1.bit7 | byte2.bit7,
                byte1.bit8 | byte2.bit8)
end

function Base.:&(byte1::Byte, byte2::Byte)
    return Byte(byte1.bit1 & byte2.bit1,
                byte1.bit2 & byte2.bit2,
                byte1.bit3 & byte2.bit3,
                byte1.bit4 & byte2.bit4,
                byte1.bit5 & byte2.bit5,
                byte1.bit6 & byte2.bit6,
                byte1.bit7 & byte2.bit7,
                byte1.bit8 & byte2.bit8)
end

We can now perform almost arbitrary logic on bits and bytes!

In [23]:
byte1 = Byte(Zero(), Zero(), One(),  Zero(), Zero(), One(), One(),  One())
byte2 = Byte(Zero(), One(),  Zero(), Zero(), Zero(), One(), Zero(), One())

Byte{Zero,One,Zero,Zero,Zero,One,Zero,One}(Zero(), One(), Zero(), Zero(), Zero(), One(), Zero(), One())

In [24]:
byte1 | byte2

Byte{Zero,One,One,Zero,Zero,One,One,One}(Zero(), One(), One(), Zero(), Zero(), One(), One(), One())

In [25]:
byte1 & byte2

Byte{Zero,Zero,Zero,Zero,Zero,One,Zero,One}(Zero(), Zero(), Zero(), Zero(), Zero(), One(), Zero(), One())

In [26]:
# Let's see if type inference is doing the computation at compile-time

@code_typed byte1 | byte2

CodeInfo(
1 ─     return $(QuoteNode(Byte{Zero,One,One,Zero,Zero,One,One,One}(Zero(), One(), One(), Zero(), Zero(), One(), One(), One())))
) => Byte{Zero,One,One,Zero,Zero,One,One,One}

In [27]:
# We can check if any logic at all is performed at run time

@code_native byte1 | byte2

	.text
; ┌ @ In[22]:4 within `|'
	movq	%rsi, -8(%rsp)
	movabsq	$140676879521816, %rax  # imm = 0x7FF1E36E9818
	retq
; └


In [28]:
# This is just a pointer to the singleton object - a compile-time constant

unsafe_load(Base.unsafe_convert(Ptr{Ptr{Nothing}}, pointer_from_objref(Byte{Zero,One,One,Zero,Zero,One,One,One}) + 0x28)) # The 0x28 comes from the layout of DataType on x64

Ptr{Nothing} @0x00007ff1e36e9818

In [29]:
## EXERCISE

# What other operations could you do in the type domain? Use your imagination!



In [30]:
# There is a type in `Base` called `Val`
# It lets you carry (immutable) data as a type parameter
# You can then pass this data between functions as compile-time information.
# WARNING: Only fast if the data is a compile-time constant!!

Val(1)

Val{1}()

### Turing Complete

To make this Turing-complete, we need some way of creating "memory". One could use more `struct`s to build a heirarchy of words and pages of increasing number of bits, complete with a system of pointers and so-on!

Alternatively, one can use the built-in `Tuple` type which can accept an arbitrary number of fields of different types. In either case, it is possible to perform just about any computation purely in the type domain.

### Practical limitations on inference: the halting problem

Julia is a friendly, interactive proramming environment. Code is being compiled and generated as users enter commands at the REPL or even during the normal execution of programs.

To avoid horrible run-time crashes of the compiler itself, type inference must protect itself from doing arbitrarily complex logic which may never halt (or take too much RAM). It is quite easy to write a program which "works" but never ends - but it is not easy to determine exactly which programs will terminate without some level of approximation.

In [31]:
# A function that never ends

never_ending_function(i::Int) = never_ending_function(i + 1)

never_ending_function (generic function with 1 method)

In [32]:
# I wonder what this does?

never_ending_function(1)

StackOverflowError: StackOverflowError:

### Type widening

To deal with this challenge, the inference engine will only work with limited precision. In situations that may be potentially dangerous for the compiler - for example involving extremely complex types, or certain kinds of recursive function calls - the inference engine will intentially "approximate" its knowledge of the types to limit compile-time complexity. Inference may assign types "wider" than those deemed possible, but never narrower.

This is an important feature of Julia's compiler and is rarely an issue in practice, if you know how to work with the compiler. 

### So, what now?

The example above is a poor use of Julia's dispatch capabilities. Next we'll discuss some more useful dispatch scenarios you use every day in Julia.

In [33]:
## QUESTIONS?

---

# Promotion

Julia's `Base` library makes use of a *promotion system* so that working with and mixing together data of differnt types is simple and easy. For example, adding an integer to a floating point number automatically gives a floating point number:

In [34]:
1 + 3.14159

4.14159

The promotion system is generic and fully-extensible to user-defined types. To promote two objects to the same type, we use the `promote` function, which returns a tuple with the two promoted objects:

In [35]:
promote(1, 3.14159)

(1.0, 3.14159)

Like many functions in `Base`, when we invoke `+` with different types, promotion will be automatically invoked. Check out this method:

```julia
+(x::Number, y::Number) = +(promote(x,y)...)
```

In [36]:
# We can see this as a lowered expression, as well

@code_lowered +(1, 3.14159)

CodeInfo(
1 ─ %1 = (Base.promote)(x, y)
│   %2 = (Core._apply)(Base.:+, %1)
└──      return %2
)

But how does `promote` work? For users, it is simple really - one just defines a `promote_rule` between two types. For example, one might define:

```julia
promote_rule(::Type{Float64}, ::Type{Int}) = Float64
```

One can do this with user-defined types and promotion will work automatically with many of `Base`s included functions.

In [ ]:
## EXERCISE: enable addition for Bits - Zero and One

Base.promote_rule(::Zero, ::One) = ...

In [ ]:
# What is One + Zero?



In [ ]:
# What is One + 3? (you might need more promote_rules...)



In [ ]:
# What is Zero + Zero?



In [ ]:
# What is One + One?



This is a really neat and easy-to-use interface for users and library developers to hook into!

But - how does it **really** work?

### Simplified promotion

To avoid a few details (error handling, efficiently promoting more than two objects, etc), I'll demonstrate a simplified version called `promote2`:

In [37]:
function promote2(a, b)
    T = promote_type2(typeof(a), typeof(b))
    return (convert(T, a), convert(T,b))
end

promote2 (generic function with 1 method)

In [38]:
# If two types are the same, no rule is needed
promote_type2(::Type{T}, ::Type{T}) where {T} = T

function promote_type2(::Type{T1}, ::Type{T2}) where {T1, T2}
    T3 = promote_rule(T1, T2)

    # It is OK if the user specified the rule one way but not the other
    if T3 === Union{}
        return promote_rule(T2, T1)
    else
        return T3
    end
end

promote_type2 (generic function with 2 methods)

In [39]:
promote2(1, 3.14159)

(1.0, 3.14159)

In [40]:
@code_typed promote2(1, 3.14159)

CodeInfo(
1 ─ %1 = (Base.sitofp)(Float64, a)::Float64
│   %2 = (Core.tuple)(%1, b)::Tuple{Float64,Float64}
└──      return %2
) => Tuple{Float64,Float64}

Promotion is a simple yet effective example of metaprogramming that

 1. Is friendly for end-users
 2. Is generic and extensible
 3. Has zero run-time overhead

These magic combination of factors is what we'll be chasing for the remainder of the workshop!

In [ ]:
## QUESTIONS?

---

# Method inlining: Faster than C?

In many languages, there is a run-time cost of adding many layers of indirection and dispatch.

Julia's compiler will automatically *inline* certain functions, when:

 * The function body is small, such that the run-time cost of a new stack frame is signficant in comparison.
 * The function has no side effects and returns a type or a constant that can be inferred. In this case the function call is replaced by the type or constant.
 
These pragmatic choices generally lead to good performance.

The automatic behavior can be controlled with the macros `@inline` and `@noinline`. There are certain situations where you might like to use these

 * Using `@inline` can force some computations and simplifications to occur at compile time, and may be useful for your metaprogramming constructs. 
 * Occassionally, using `@noinline` may help you avoid large function bodies or harmful "optimizations" that may result in worse performance.
 * `@noinline` is useful for creating benchmarking code (avoiding the situation where the compiler may elide what you are trying to measure).
 * Micro-optimizations

In [41]:
# A version of + that uses promote2

add2(x,y) = +(promote2(x,y)...)

add2 (generic function with 1 method)

In [42]:
# promote2, promote_type2, promote_rule and convert are all inlined and optimized away

@code_typed add2(1, 3.14159)

CodeInfo(
1 ─ %1 = (Base.sitofp)(Float64, x)::Float64
│   %2 = (Base.add_float)(%1, y)::Float64
└──      return %2
) => Float64

Method inlining can make a significant difference to performance. Traditional C and C++ compiler implementations often perform inlining as a "link-time optimization" (LTO) where generated code is inlined. Julia performs inlining at a higher level of IR where a greater amount of semantic information about the code is retained, leading to further optimizations opportunities (we will discuss the interplay between inlining, constant propagation and inference later).

In [43]:
using BenchmarkTools

In [44]:
@noinline function promote3(a, b)
    T = promote_type2(typeof(a), typeof(b))
    return (convert(T, a), convert(T,b))
end

add3(x,y) = +(promote3(x,y)...)

add3 (generic function with 1 method)

In [45]:
@benchmark add2($1, $3.14159)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     0.017 ns (0.00% GC)
  median time:      0.022 ns (0.00% GC)
  mean time:        0.023 ns (0.00% GC)
  maximum time:     0.170 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

In [46]:
@benchmark add3($1, $3.14159)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     0.017 ns (0.00% GC)
  median time:      0.021 ns (0.00% GC)
  mean time:        0.020 ns (0.00% GC)
  maximum time:     0.055 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

In [ ]:
## QUESTIONS?

---
#  Games with tuples: splatting, slurping and recursion

Some functions in Julia accept a variable number of arguments - so called "varags" functions.

In [47]:
+(1,2,3,4,5,6,7,8,9,10)

55

### Slurping

A function can "slurp" up its (tail) arguments into a tuple using the `...`.

(Note that `...` is also used for the inverse operation, called "splatting", which we will discuss below.)

In [49]:
# This function "slurps" up all of it's inputs and places them into a tuple called x
slurp(x...) = x

slurp(1,2,3)

(1, 2, 3)

In [51]:
# We can also specify the type of the variable arguments
slurp(x::Integer...) = x

# In this case, the ... is syntactic sugar for a Vararg type
slurp(x::Vararg{Integer}) = x

# You can even specify the number of arguments as the second type parameter to Vararg
# Frequently this will be free parameter (but it can also be a constant)
slurp(x::Vararg{Integer, N}) where {N} = x

slurp (generic function with 2 methods)

In [52]:
## EXERCISE

# Explore the different signatures of `slurp` we just created, using `methods`

methods(slurp)

# 2 methods for generic function "slurp":
[1] slurp(x::Vararg{Integer,N}) where N in Main at In[51]:9
[2] slurp(x...) in Main at In[49]:2

The function signature can contain other arguments before the varargs.

```julia
map(f, x1, x2, xs...)
```

### What is a method signature, anyway?

A generic function can have many methods.

Each method has a signature, which is represented by a (possibly abstract) `Tuple` type, like `Tuple{Integer, Integer}`. The `Tuple` type is unique in Julia in that it's parameters are *covariant* - for example `Tuple{Int64, Int32}` is a subtype of `Tuple{Integer, Integer}`.

By default, a method is invoked with a precise (concrete) set of types, like `Tuple{Int64, Int32}`. These **specialized** methods are indivually optimized, inferred and compiled to machine code.

### `Varargs` and `NTuple`

We can also talk about `Tuple`s of variable length, most commonly known through the `NTuple{N, T}` type. Sometimes this is just convenient shorthand, like `NTuple{3, Float64}` is shorter to write than `Tuple{Float64, Float64, Float64}`.

In other cases we want to be able to accept tuples of variable lengths, sometimes with restrictions like

```julia
map(f, t1::NTuple{N}, t2::NTuple{N}) where {N}
```
which accepts any two tuples so long as they have the same length.

Fun fact:

> The `NTuple{N, T}` type is actually a simple *alias* of the type `Tuple{Vararg{T, N}}`!

This provides a direct link between a varargs function signatures such as `f(::Integer...)` and the signature type `Tuple{Vararg{Integer}}`.

In [53]:
NTuple{3, Integer}

Tuple{Integer,Integer,Integer}

In [54]:
Tuple{Vararg{Integer, 3}}

Tuple{Integer,Integer,Integer}

In [55]:
Tuple{Vararg{Integer}}

Tuple{Vararg{Integer,N} where N}

In [56]:
# Like a function signature, their may be other types before the varargs

Tuple{String, Symbol, Float64, Vararg{Integer}}

Tuple{String,Symbol,Float64,Vararg{Integer,N} where N}

### Splatting

One can perform the reverse operation, and "splat" a variable number of arguments into a function:

In [57]:
x = (2, 3)
*(x...)

6

Similarly, we can splat out arguments into tuples, such as

```julia
append(t1::Tuple, t2::Tuple) = (t1..., t2...)
```

**Note:** Splatting is not limited to tuples - it is a generic iterator concept, relying on `iterate`, and can occur in other contexts like array literals, `[x

Combining splatting and slurping can be very powerful. For example:

In [90]:
# `head` returns the first element of a tuple

head(t::Tuple) = t[1]

head (generic function with 1 method)

In [91]:
## EXERCISE

# Create a `tail` function that returns all but the first argument of a tuple
# Hint: begin with `tail(x::Tuple) = _tail(x...)` and define a suitable method for `_tail`.

In [84]:
_tail(_, x...) = x
tail(x::Tuple) = _tail(x...)

tail (generic function with 1 method)

In [85]:
tail((1,2,3))

(2, 3)

### Recursion with tuples

Splatting and slurping can be used to perform various recursive algorithms over tuples.

Take for example a possible implementation of `map` (called `map2`). In this pattern we split appart out tuple using splatting, and work on it element-by element recursively using an inner helper function called `_map2`.

In [156]:
?===

search: === == !==



```
===(x,y) -> Bool
≡(x,y) -> Bool
```

Determine whether `x` and `y` are identical, in the sense that no program could distinguish them. First the types of `x` and `y` are compared. If those are identical, mutable objects are compared by address in memory and immutable objects (such as numbers) are compared by contents at the bit level. This function is sometimes called "egal". It always returns a `Bool` value.

# Examples

```jldoctest
julia> a = [1, 2]; b = [1, 2];

julia> a == b
true

julia> a === b
false

julia> a === a
true
```


In [58]:
map2(f, t::Tuple) = _map2(f, t...)

# If the tuple is empty we return an empty tuple
_map2(f) = ()

# Otherwise we map the first element and apply _map to the remainder
_map2(f, x, y...) = (f(x), _map2(f, y...)...)

_map2 (generic function with 2 methods)

In [59]:
map2(-, (1,2,3))

(-1, -2, -3)

There may be more than one way to write this function using recursive methods.

The advantage of the method shown is that each subsequent call to `_map2` is getting simpler and simpler until `_map2(f)` is called.

When recursive function calls are made and are recursively becoming *more* complex, type inference may invoke widening to protect itself, resulting in sub-optimal code. It is recommended to take great care when using recursive methods.

Next we do some excersises! Next, use a similar pattern to perform a reduction over a tuple

In [60]:
## EXERCISE

# Write a function `sum2` which adds all the elements of the tuple.

sum2(t::Tuple) = +(t...)

sum2 (generic function with 1 method)

In [61]:
# test case
sum2((1,))

1

In [62]:
# test case
sum2((1,2,3,4,5))

15

In [23]:
## EXTENSION EXERCISE

# Create a more generic `reduce2` (or `mapreduce2`) function that works recursively over a tuple
function reduce2(f, t::Tuple)
    _reduce2(f, t...)
end

_reduce2(f, h, t) = f(h, t)
_reduce2(f, h, t...) = _reduce2(f, h, _reduce2(f, t...))

_reduce2 (generic function with 2 methods)

In [25]:
methods(_reduce2)

# 2 methods for generic function "_reduce2":
[1] _reduce2(f, h, t) in Main at In[23]:8
[2] _reduce2(f, h, t...) in Main at In[23]:9

In [24]:
# test case
reduce2(*, (1,2,3,4,5))

120

In [99]:
## EXTENSION EXERCISE

# Create a method for `map2` which works on 2 input tuples, like `map2(+, (1,2,3), (4,5,6)) == (5,7,9)`
# Hint: one possible solution would take advantage of the `head` and `tail` functions written earlier...

function map2(f, t1::Tuple{Vararg{N}}, t2::Tuple{Vararg{N}}) where N  # N ensures tuples are of same length
    f(head(t1), head(t2)), map2(f, tail(t1), tail(t2))...
end

map2(f, t1::Tuple{Vararg{0}}, t2::Tuple{Vararg{0}}) = ()

map2 (generic function with 4 methods)

In [100]:
map2(+,(), ())

()

In [101]:
# test case
map2(+, (1,2,3), (4,5,6))

(5, 7, 9)

### Inlining and recursion

Method inlining plays an important role in recursive algorithms.

Without inlining, a recursive algorithm on a tuple of length `N` is *O*(`N^2`). In the intermediate stages, *O*(`N`) tuples with average length *O*(`N`) are constructed and passed as arguments to the next function. The overhead in creating all those stack frames is enormous.

In general - this kind of recursion is a **terrible algorithm**!

In Julia, we sometimes use this pattern because combined with inlining and compiler optimizations to handle slurping and splatting, the run-time cost is only *O*(`N`) and this can be a convenient way to deal with data of different types at maximum speed. However - it is still may be hard work for the compiler.

In [102]:
@benchmark map2(-, $(ntuple(identity, 3)))

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.837 ns (0.00% GC)
  median time:      1.909 ns (0.00% GC)
  mean time:        1.957 ns (0.00% GC)
  maximum time:     12.390 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

In [103]:
@benchmark map2(-, $(ntuple(identity, 32)))

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     5.180 ns (0.00% GC)
  median time:      5.186 ns (0.00% GC)
  mean time:        5.350 ns (0.00% GC)
  maximum time:     16.686 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

In [104]:
@benchmark map2(-, $(ntuple(identity, 33)))

BenchmarkTools.Trial: 
  memory estimate:  10.03 KiB
  allocs estimate:  98
  --------------
  minimum time:     46.496 μs (0.00% GC)
  median time:      51.410 μs (0.00% GC)
  mean time:        60.121 μs (13.16% GC)
  maximum time:     52.655 ms (99.87% GC)
  --------------
  samples:          10000
  evals/sample:     1

If you're not careful - there is a huge performance cliff!

With more than 32 elements, inference gives up on tracking the size of the tuples, and inlining the recursive function calls is disabled.

For more complicated recursive functions, you may also want to invoke `@inline` to force the recursion to unroll (which will only work so long as the types are fully inferred).

In [ ]:
## QUESTIONS?

---
# Constant propagation - and what are `@pure` functions?

One of the optimizations performed by the compiler is "constant" propagation.

### What's a constant?

Values are "constant" if they are known to the compiler. These include

 * Literals like `true`, `1`, `3.0` or `:symbol`
 * Any known type
 * Any known type parameter
 * Any singleton object of a known type (i.e. a `struct` with no fields)
 * Other constants determined by the compiler
 
The compiler can determine constants from other constants in a number of ways

 * Tuples can be constructed from constants, `t = (a, b)` where `a` and `b` are known.
 * Constant tuples can be indexed, splatted or slurped with other constants, `t[1]`, `t2 = (t..., c)`, etc.
 * Getting a field of a constant `struct`
 * Functions that are computable from the type signature alone, like `length(::Tuple{Vararg{Any, N}}) where {N} = N`
 * Functions that are marked `@pure` and have constant inputs.
 * `===` between two constants or known types
 * Some other in-built functions that have constant inputs, like `Base.add_int(1, 2)`
 * ...
 
This is a growing list, as the compiler evolves and improves.

In [105]:
# A simple function where we can infer the output

constant_function() = 1 + 2

constant_function (generic function with 1 method)

In [106]:
# What does the compiler do?

@code_typed constant_function()

CodeInfo(
1 ─     return 3
) => Int64

In [107]:
# Constant propagation is particularly useful for collapsing nested calls of functions

outer_function() = constant_function() + constant_function()

outer_function (generic function with 1 method)

In [108]:
# This is essentially a form of inlining

@code_typed outer_function()

CodeInfo(
1 ─     return 6
) => Int64

In [109]:
# Even if the input is not constant, the output might be!

constant_function2(x) = 1 + 2

constant_function2 (generic function with 1 method)

In [110]:
@code_typed constant_function2(x)

CodeInfo(
1 ─     return 3
) => Int64

In [111]:
# Even if the function is impure, it's return value might be known by the compiler

function impure_constant_function(x)
    x[1] = 1 + 2
    return 1 + 2
end

impure_constant_function (generic function with 1 method)

In [112]:
@code_typed impure_constant_function([1,2,3])

CodeInfo(
1 ─     (Base.arrayset)(true, x, 3, 1)::Array{Int64,1}
└──     return 3
) => Int64

In [113]:
# Not all operations will propagate values automatically

not_constant_function() = 1 + pi

not_constant_function (generic function with 1 method)

In [114]:
# Here we see that floating-point addition will not propagate constants by default
# NOTE: Probably a good thing!

@code_typed not_constant_function()

CodeInfo(
1 ─     return 4.14159
) => Float64

### Constants and type parameters

Types and their parameters are also compile-time constant values.

This works both ways

 * Extracting a type parameter results in a constant
 * Type inference can only succeed when type parameters are constant values (or known types)

In [116]:
# Type parameters can be inferred from constant values

impure_constant_function2(x) = Val(impure_constant_function(x))

impure_constant_function2 (generic function with 1 method)

In [117]:
@code_typed impure_constant_function2([1,2,3])

CodeInfo(
1 ─     (Base.arrayset)(true, x, 3, 1)::Array{Int64,1}
└──     return $(QuoteNode(Val{3}()))
) => Val{3}

In [119]:
?Val

search: Val values valtype eval evalfile @eval isvalid @evalpoly



```
Val(c)
```

Return `Val{c}()`, which contains no run-time data. Types like this can be used to pass the information between functions through the value `c`, which must be an `isbits` value. The intent of this construct is to be able to dispatch on constants directly (at compile time) without having to test the value of the constant at run time.

# Examples

```jldoctest
julia> f(::Val{true}) = "Good"
f (generic function with 1 method)

julia> f(::Val{false}) = "Bad"
f (generic function with 2 methods)

julia> f(Val(true))
"Good"
```


In [120]:
# A type parameter that depends on a variable

not_const_function2(x) = Val(x)

not_const_function2 (generic function with 1 method)

In [121]:
# There's no way to fully determine the type from variable `x`

@code_typed not_const_function2(3)

CodeInfo(
1 ─ %1 = invoke Main.Val(_2::Int64)::Val{_1} where _1
└──      return %1
) => Val{_1} where _1

In [ ]:
## QUESTIONS?

### `@pure` functions

A function can be marked as `@pure` as an assertion to the compiler that it may be invoked at compiler-time with no side effects.

Note that `Base.@pure` is not exported by default. If you need it you can `import` it or specify it fully.

In [122]:
# Mark this function as `@pure`

Base.@pure pure_function(x) = 1 + pi

pure_function (generic function with 1 method)

In [123]:
# `@pure` functions behave like normal functions...

@code_typed pure_function(1)

CodeInfo(
1 ─     return 4.14159
) => Float64

In [124]:
# ... unless they are called from another function with constant inputs

call_pure_function() = pure_function(1)

@code_typed call_pure_function()

CodeInfo(
1 ─     return 4.14159
) => Float64

**Beware**

There are severe limitations on the capabilities of `@pure` functions!

Earlier we said that `@pure` is

> an assertion to the compiler that it may be invoked at compiler-time with no side effects.

In Julia, many statements have side effects on the state of the compiler itself. Every time a new type or method is defined, the state of the compiler changes to reflect the new information. The current state of the type system and method tables is called the "world". Every time the world changes, the "world age" increments. If methods are overwritten or specialized, other methods may also change.

The compiler will assume that a `@pure` function will not change *in any way* as the world age is incremented. It will not be recompiled to deal with new methods and you will encounter a "world age error" if it attempts to perform dispatch based on new types.

Thus - `@pure` functions are **not suitable for use in most situations**. It may be safe in cases where the input types are fully specified and the dependent function calls are very basic. In most situations it is better to use constant propagation.

***You have been warned!***

### Inter-procedural constant propagation

Typically, functions (not marked as `@pure`) are specialized on their input types alone. Constants will only propagate within the scope of the method (or as a return value, if it can be evaluated).

Sometimes, it is useful to optimize the cases where some but not all the inputs are constant. Julia indirectly supports this by supporting constant propagation through function barriers when the function is inlined.

Enforcing inlining to allow constant propagation may be a valid use for `@inline`. This is particularly useful if the return type depends on an input parameter:

In [ ]:
@inline function firstchar(str::String, return_char::Bool = true)
    if return_char
        return str[1]
    else
        return String(str[1])
    end
end

### `getproperty` overloading

New to Julia v0.7 is the ability to overload the `getproperty` function, where `a.b` is syntactic sugar for `getproperty(a, :b)`. The default definition is similar to:

```julia
getproperty(x, f::Symbol) = getfield(x, f)
```
where `getfield` accesses directly a field in a `struct` or `mutable struct`.

`getproperty` relies on constant propagation for efficiency.

In [133]:
## EXERCISE

# Define `getproperty` for `Array` so that we can get it's length or size via `a.length` or `a.size`.

@inline function Base.getproperty(a::Array, s::Symbol)
    getfield(a, s)
end

In [134]:
# Check if the return type can be correctly inferred

function get_length(a::Array)
    return a.length
end

@code_typed get_length([1,2,3])

CodeInfo(
1 ─     (Main.getfield)(a, :length)::Union{}
│       $(Expr(:unreachable))::Union{}
│       φ ()::Union{}
└──     $(Expr(:unreachable))::Union{}
) => Union{}

In Julia, it would not be usual to define complex properties in this way.

Generally, I feel this is useful for:

 * defining a consistent *interface* across different related types (which might have different `struct` fields)
 * making types similar to `NamedTuple` which act as if they have user-definable field names

---

# Dispatch revisited: Interfaces and Traits

Programming in Julia is organized around a tree of data types and a variety of extensible, generic functions.

Generic functions can be overloaded with any number of methods. For some functions there is only one method, for others there are very many.

In [135]:
methods(pwd) # prints working directory

# 1 method for generic function "pwd":
[1] pwd() in Base.Filesystem at file.jl:51

In [136]:
methods(+) # Add's two or more objects

# 163 methods for generic function "+":
[1] +(x::Bool, z::Complex{Bool}) in Base at complex.jl:277
[2] +(x::Bool, y::Bool) in Base at bool.jl:104
[3] +(x::Bool) in Base at bool.jl:101
[4] +(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:112
[5] +(x::Bool, z::Complex) in Base at complex.jl:284
[6] +(a::Float16, b::Float16) in Base at float.jl:392
[7] +(x::Float32, y::Float32) in Base at float.jl:394
[8] +(x::Float64, y::Float64) in Base at float.jl:395
[9] +(z::Complex{Bool}, x::Bool) in Base at complex.jl:278
[10] +(z::Complex{Bool}, x::Real) in Base at complex.jl:292
[11] +(::Missing, ::Missing) in Base at missing.jl:96
[12] +(::Missing) in Base at missing.jl:83
[13] +(::Missing, ::Number) in Base at missing.jl:97
[14] +(level::Base.CoreLogging.LogLevel, inc::Integer) in Base.CoreLogging at logging.jl:106
[15] +(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:406
[16] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:434
[17] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:433
[18] +(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:432
[19] +(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:403
[20] +(x::BigInt, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.GMP at gmp.jl:440
[21] +(x::BigInt, c::Union{Int16, Int32, Int64, Int8}) in Base.GMP at gmp.jl:446
[22] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:556
[23] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:549
[24] +(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:543
[25] +(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:402
[26] +(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:371
[27] +(x::BigFloat, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.MPFR at mpfr.jl:378
[28] +(x::BigFloat, c::Union{Int16, Int32, Int64, Int8}) in Base.MPFR at mpfr.jl:386
[29] +(x::BigFloat, c::Union{Float16, Float32, Float64}) in Base.MPFR at mpfr.jl:394
[30] +(x::Dates.CompoundPeriod, y::Dates.CompoundPeriod) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.1/Dates/src/periods.jl:343
[31] +(x::Dates.CompoundPeriod, y::Dates.Period) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.1/Dates/src/periods.jl:341
[32] +(x::Dates.CompoundPeriod, y::Dates.TimeType) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.1/Dates/src/periods.jl:371
[33] +(x::Dates.Date, y::Dates.Day) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:74
[34] +(x::Dates.Date, y::Dates.Week) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:72
[35] +(dt::Dates.Date, z::Dates.Month) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:55
[36] +(dt::Dates.Date, y::Dates.Year) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:28
[37] +(dt::Dates.Date, t::Dates.Time) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:19
[38] +(t::Dates.Time, dt::Dates.Date) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:20
[39] +(x::Dates.Time, y::Dates.TimePeriod) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:78
[40] +(dt::Dates.DateTime, z::Dates.Month) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:48
[41] +(dt::Dates.DateTime, y::Dates.Year) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:24
[42] +(x::Dates.DateTime, y::Dates.Period) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:76
[43] +(B::BitArray{2}, J::LinearAlgebra.UniformScaling) 

In [137]:
## EXERCISE

# View the methods of some function you're interested in

methods(sin)

# 12 methods for generic function "sin":
[1] sin(x::BigFloat) in Base.MPFR at mpfr.jl:736
[2] sin(::Missing) in Base.Math at math.jl:1070
[3] sin(a::Complex{Float16}) in Base.Math at math.jl:1019
[4] sin(a::Float16) in Base.Math at math.jl:1018
[5] sin(z::Complex{T}) where T in Base at complex.jl:795
[6] sin(x::T) where T<:Union{Float32, Float64} in Base.Math at special/trig.jl:30
[7] sin(x::Real) in Base.Math at special/trig.jl:53
[8] sin(A::LinearAlgebra.Hermitian{#s622,S} where S<:(AbstractArray{#s623,2} where #s623<:#s622) where #s622<:Complex) in LinearAlgebra at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.1/LinearAlgebra/src/symmetric.jl:728
[9] sin(A::Union{Hermitian{#s623,S}, Symmetric{#s623,S}} where S where #s623<:Real) in LinearAlgebra at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.1/LinearAlgebra/src/symmetric.jl:724
[10] sin(D::LinearAlgebra.Diagonal) in LinearAlgebra at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.1/LinearAlgebra/src/diagonal.jl:468
[11] sin(A::AbstractArray{#s623,2} where #s623<:Real) in LinearAlgebra at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.1/LinearAlgebra/src/dense.jl:788
[12] sin(A::AbstractArray{#s623,2} where #s623<:Complex) in LinearAlgebra at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.1/LinearAlgebra/src/dense.jl:795

## What is an interface?

In theory, the methods of a function do not have to share any semantic similarity. From the language's perspective, there's nothing **wrong** with defining:

In [138]:
add(x::Int, y::Int) = x + y
add(x::Float64, y::Float64) = x - y

add (generic function with 2 methods)

However - clearly this is a **bad** idea!

This is because humans need to be able to read, reason about and write code - no matter the programming language or its rules. If you do this in the middle of your company's large piece of software, some poor programmer will get very confused, and your coworkers will get rather upset with you!

Very loosely defined, when I say "interface" I mean the set of language and semantic guarantees surrounding how one or more generic functions behave for different sets of input types.

Later, I'll define more precisely a concept of "strong interface" with the aim of defining what these guarantees exactly should be.

But for now let's accept the following as good practice:

> Given a generic function `f`, the meaning of `f(x::T)` should be consistent and predictable accross different types `T`.

If this were not true, then a generic function `g` (which accepts a range of input types) would not be able to safely and reliably call `f` for all of *its* input types.

And of course, the writer of `g` doesn't want it to crash for its users!

**Case study:** `hash` and `isequal` 

In [139]:
?hash

search: hash hasmethod haskey skipchars Threads BenchmarkTools MathConstants



```
hash(x[, h::UInt])
```

Compute an integer hash code such that `isequal(x,y)` implies `hash(x)==hash(y)`. The optional second argument `h` is a hash code to be mixed with the result.

New types should implement the 2-argument form, typically by calling the 2-argument `hash` method recursively in order to mix hashes of the contents with each other (and with `h`). Typically, any type that implements `hash` should also implement its own `==` (hence `isequal`) to guarantee the property mentioned above. Types supporting subtraction (operator `-`) should also implement [`widen`](@ref), which is required to hash values inside heterogeneous arrays.


In [140]:
?isequal

search: isequal issetequal



```
isequal(x, y)
```

Similar to [`==`](@ref), except for the treatment of floating point numbers and of missing values. `isequal` treats all floating-point `NaN` values as equal to each other, treats `-0.0` as unequal to `0.0`, and [`missing`](@ref) as equal to `missing`. Always returns a `Bool` value.

# Implementation

The default implementation of `isequal` calls `==`, so a type that does not involve floating-point values generally only needs to define `==`.

`isequal` is the comparison function used by hash tables (`Dict`). `isequal(x,y)` must imply that `hash(x) == hash(y)`.

This typically means that types for which a custom `==` or `isequal` method exists must implement a corresponding `hash` method (and vice versa). Collections typically implement `isequal` by calling `isequal` recursively on all contents.

Scalar types generally do not need to implement `isequal` separate from `==`, unless they represent floating-point numbers amenable to a more efficient implementation than that provided as a generic fallback (based on `isnan`, `signbit`, and `==`).

# Examples

```jldoctest
julia> isequal([1., NaN], [1., NaN])
true

julia> [1., NaN] == [1., NaN]
false

julia> 0.0 == -0.0
true

julia> isequal(0.0, -0.0)
false
```

---

```
isequal(x)
```

Create a function that compares its argument to `x` using [`isequal`](@ref), i.e. a function equivalent to `y -> isequal(y, x)`.

The returned function is of type `Base.Fix2{typeof(isequal)}`, which can be used to implement specialized methods.


# Strong Interface

Unlike languages such as Go, Haskell and others, Julia doesn't have a *formal* way of specifying an interface. It is a gentlemen's agreement - or quite often a bit more like a pirate's accord.

Take for example `AbstractArray{T, N}`. When I perform scalar `getindex` on any `AbstractArray{T}` I expect that I get a `T` back.

In [141]:
array = [1, 2, 3]

array[1] isa eltype(array)

true

If I were to create my own array type that *lied* about it's element type, that would

 1. Be bad. Many functions would crash.
 2. Be perfectly allowable according to the formal rules of Julia.

In order to have our library functions compose together and be extensible to new types, we must respect their respective interfaces. Generally, an interface is documented, like

```julia
"""
    isbla(x)

Return true if `x` is bla, and false otherwise.
"""
isbla(::Any) = false
isbla(i::Integer) = iseven(i)
isbla(::Complex) = true
```

What can we say about this function? Well, for one thing, it better return a `Bool`, since the documentation says it can return `true` or `false`. If you can figure out from the description what "bla" might mean, you might overload it for your own type.

However, this still poses difficulty in some situations - what should `isbla(missing)` be? Given this docstring, users are likely to write code such as `if isbla(x); ...; else; ...; end`. However, if `isbla(::Missing) = missing` then this code will crash!

To me, what I call a *strong* interface is one which favours **blind composition**. A library writer "Alice" is creating a method `f(::A)` and knows about a method `g(::A)`. A second author "Bob" creates type `B <: A` and creates a specialized method for `g(::B)`. To support blind composition, Bob's method `g(::B)` must satisfy everything about `g(::A)` that Alice can reasonably assume about its behavior and output. Generally, this might include:

 * Guarantees on the output type of `g(::A)` should be respected by `g(::B)`. This can be an abstract type - for example, `similar(::AbstractArray)` should always be an `AbstractArray`, or blind composition will not work.
 * Any semantic understanding of the behavior of `g(::A)`. We shouldn't write a method for `map` that only transforms the first element!
 * How `g` mutates it's input or has (relevant) side-effects. We shouldn't write a method for `map!` that only mutates it's first element!

In [ ]:
# This is all bad! Bad, bad, BAD!

struct BadArray <: AbstractMatrix{Int}
   data::Vector{Float64}
end

Base.getindex(a::BadArray, i...) = a.data[i...]
Base.axes(a::BadArray) = axes(a.data)
Base.size(a::BadArray) = size(a.data)

Base.similar(a::BadArray) = Dict()
Base.map!(f, a::BadArray) = (a[1] = f(a[1]))

In [ ]:
## EXERCISE

# Try to do literally *anything* with BadArray!

BadArray([1.0, 2.5, 3.14])

In conclusion, the most important feature of well designed interfaces is that they can be

 * extended
 * composed
 
An interface might be extended by e.g. creating a subtype of `AbstractArray` that correctly defines `getindex`, `axes` and `size`.

In [ ]:
struct Vec3{T} <: AbstractVector{T}
    data::Tuple{Float64, Float64, Float64}
end

Vec3(x::T, y::T, z::T) where {T} = Vec3{T}((x, y, z))

Base.getindex(v::Vec3, i::Int) = v.data[i]
Base.size(::Vec3) = (3,)
Base.axes(::Vec3) = (Base.OneTo(3),)

In [ ]:
vec3 = Vec3(1.0, 2.5, 3.0)

### Traits

Traits are properties of types that can be determined at compile-time and can be used to aid dispatch.

One prominent example of a trait is `IndexStyle`, which controls if a multidimensional array will be more quickly indexed by a single, linear index (as if it were a dense array) or a set of Cartesian indices. In this case we can define the trait as

```julia
Base.IndexStyle(::Type{<:Vec3}) = IndexLinear()
```

There are multiple ways of defining indices, but one recommended pattern (used by `Base`) is:

 * Each trait class has a common abstract supertype, in this case `IndexStyle`
 * A trait can be determined by calling the constructor for the trait class with a type.
 * The returned trait is an instance, in this case `IndexLinear()`.
 
Following this, one can construct a fairly simple dispatch pattern that takes account of the traits. In this case, this is *approximately*:

```julia
function getindex(a::AbstractArray, inds...)
    _getindex(IndexStyle(typeof(a)), a, to_indices(a, inds)...)
end
```
where `to_indices` expands any `:` to an index range for the relevant dimension.

Now, we can define seperate methods for
```julia
function _getindex(::IndexLinear, a::AbstractArray, inds...)
    ...
end
```
and a different method for
```julia
function _getindex(::IndexCartesian, a::AbstractArray, inds...)
    ...
end
```
which may be faster for those kinds of arrays.

In the above, the trait dispatch allows us to perform different implementations based on some property of the `AbstractArray`. Doing it this way has the following benefits:

 * `getindex` itself gives the same answer, not matter the array type.
 * An end user doesn't need to worry about the traits, only the library writer.
 * It still isn't one-size-fits all. There is more than one generic implementation.
 * We only need to write one scalar `getindex` method for each new array. Multiple `getindex` is fully automated, meaning we don't need to rewrite a whole bunch of code - resulting in faster, less buggy code.
 * Julia packages are free to define a new subtype of `IndexStyle` if its helpful.

Let's see it in action!

In [ ]:
@code_lowered getindex(vec3, :)

In [ ]:
## QUESTIONS?

---

# Expressions and macros

Finally, we turn out attention to a more "direct" form of metaprogramming, where we directly interact with, and execute, Julia code expressions.

### What's an expression?

Well, there's *expressions*, and then there's `Expr`s.

 * Any valid piece of Julia code is an "expression".
 * All expressions have a type and a value.
 * There is no "empty expression", or `void`. The value `nothing::Nothing` is inserted by lowering when encountering empty blocks.
 * Any value can be an expression - typically literals, types or constants.
 * The name of a binding in-scope is a valid expression - consider `f(x) = x`. The expression for the right-hand-side is `:x`. The namespace of a symbol is discovered by lowering based on the scoping rules.
 * `Symbol`s can also themselves be values... for `f(x) = :x` the right-hand-side is `QuoteNode(:x)`.
 * There are a few other special expression objects, like `LineNumberNode` and so-on.
 * But, almost all complex, nested expressions are represented by the `Expr` type.
 
One can create expressions using the following syntax

```julia
:( ... )
```
or
```julia
quote
    ...
end
```
Let's explore!

In [142]:
# A literal value is still an expression - it doesn't need to be wrapped in a special wrapper type

:(1)

1

In [143]:
:( my_variable )

:my_variable

An `Expr` has a `head::Symbol` and a list of `args::Vector{Any}`.

Let's explore some common cases. It is easier to view the elements of an `Expr` by calling `dump` to lay out it out as a tree:

In [144]:
my_expr = :(a.b)

:(a.b)

In [145]:
# The expression head is the Symbol for "."

my_expr.head

:.

In [146]:
# The first element is a reference to the binding in scope called "a"
# The second element is a reference to the Symbol ":b" - the name of the field

my_expr.args

2-element Array{Any,1}:
 :a   
 :(:b)

In [147]:
# We can see it easier using `dump`
# The "b" part is wrapped in a `QuoteNode`

dump(:(a.b))

Expr
  head: Symbol .
  args: Array{Any}((2,))
    1: Symbol a
    2: QuoteNode
      value: Symbol b


In [148]:
# Some operators are directly translated to :call expressions 

dump(:(a + b))

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Symbol a
    3: Symbol b


In [149]:
# Some other operators get their own expression `head` type

dump(:(a'))

Expr
  head: Symbol '
  args: Array{Any}((1,))
    1: Symbol a


In [150]:
# quote blocks sometimes be a more convenient way of constructing an expression
# By default, they insert line information about where the quote block is in the code

quote
    x
end

quote
    #= In[150]:5 =#
    x
end

In [151]:
# At the surface level, `Expr`s with head `:block` are created by `begin ... end`

dump(quote
    x
end)

Expr
  head: Symbol block
  args: Array{Any}((2,))
    1: LineNumberNode
      line: Int64 4
      file: Symbol In[151]
    2: Symbol x


In [ ]:
## EXERCISE

# Explore some expressions you use. Examples: 
#  * if statements
#  * for loop
#  * getting and setting array elements
#  * constructing arrays
#  * constructing typed arrays

dump(:( ... ))

In [ ]:
# What does the expression look like for a macro call?


In [ ]:
## EXTENSION EXERCISE

# Directly using the `Expr` constructor, create an expression for setting the index `i` of an array `a` to value `3`.



### Creating expressions via interpolation

When metaprogramming, it is common to want to create expression programatically, where parts of the expression depend on some data you have.

It is a little annoying to have to construct expressions fully-by hand. For example:

```julia
function expr_maker(i)
    return Expr(:block, Expr(:=, Expr(:ref, :a, i), 3))
end
```

To make life easier, Julia allows interpolation of values using `$`, directly into expressions.

```julia
function expr_maker(i)
    return :( a[$i] = 3 )
end
```

It is a **lot** easier to read the second version!

In [ ]:
# Make return the expression for a vector literal `[1, 2, 3, ..., n]` for variable `n`

function make_vect1(n)
    return Expr(:vect, 1:n...)
end

make_vect1(5)

In [ ]:
# The same, using interpolation syntax

function make_vect2(n)
    return :( [$(1:n...)] )
end

make_vect2(5)

In [ ]:
## EXERCISE

# Do the same thing for tuple (1, 2, 3, ..., n)

function make_tuple(n)
    return ...
end

make_tuple(5)

### Macros

Macros are special functions that transform expressions and insert the result back into your code.

This can be useful for a variety of reasons.

 * A convenient way to "annotate" expressions, like `@inline f(x) = ...`
 * Provide tools that take expressions and use them for a task - `@time`, `@code_typed`, `@benchmark`, etc
 * Create a useful shortcut to save on typing - `Base.@nloops` makes arbitry number of nested loops
 * Create an entire DSL - like *Query.jl*, *Flux.jl*, etc.

Macros are defined with the `macro` keyword:

```julia
macro negative(ex)
    return :( - $ex )
end
```

And called with an `@` symbol:

```julia
minus_one = @negative 1
```

The `@` is a signifier to the user (and parser) that a syntax transformation is occuring.

macro negative(ex)
    return :( - $ex )
end

In [ ]:
minus_one = @negative 1

In [ ]:
x = 2
@negative x

### Macro dispatch

You can define multiple methods for a macro. However, they can only *dispatch* on the expression types.

This is most useful for defining a macro of different *arity* (number of inputs). For example

```julia
macro m(ex)
    # do something with one expression
end

macro m(ex1, ex2)
    # do something with two expressions
end
```

However, expressions do come in different types and it is perfectly valid define methods for different expression types - such as integer literals

In [152]:
macro superpower(i::Int)
    return :($i ^ $i)
end

@superpower (macro with 1 method)

In [153]:
@superpower 2

4

In [154]:
@superpower 3

27

In [155]:
# This macro expects a integer literal value, not a variable name

x = 3
@superpower x

LoadError: MethodError: no method matching @superpower(::LineNumberNode, ::Module, ::Symbol)
Closest candidates are:
  @superpower(::LineNumberNode, ::Module, !Matched::Int64) at In[152]:2

Macros also gain access to some metadata about where they are called from and from which module (the first two arguments in the method error above).

In [ ]:
## QUESTIONS?

---

# Generated functions and when (not) to use them

The final metaprogramming technique that we will cover is `@generated` functions.

Such functions represent a form of staged programming where code is created based on the input. 

To execute at full speed with Julia's compilation process, they are designed to generate code based on the *type* of the input arguments. If the code depended on run-time data, then that code would have to be compiled every time the function would be called! Thus, we are limited to the information available to type inference.

Distinct code is generated for each unique, concrete function signature. You can see the generated code using `@code_lowered`.

In [ ]:
@generated function sum3(t::NTuple{N, Any}) where {N}
    # This body is the "function generator"
    elements = [ :( t[$i] ) for i = 1:N ]
    expr = :( +($(elements...)) )
    
    return expr # The returned expression is the "generated function body"
end

In [ ]:
sum3((1, 2.0, pi))

In [ ]:
@code_lowered sum3((1, 2.0, pi))

In [ ]:
@code_lowered sum3((1, 2, 3, 4, 5, 6, 7, 8))

Please note that there are several restrictions regarding generated functions

 * The compiler provides no guarantees whether the code generation is invoked one time, many times or even zero times (because of precompilation).
 * The code generator should have no side effects and has the same limitations as `@pure` functions.
 * The generated code cannot create new types. This means you can not generate code with closures (note: it is always possible to define your own closure type). This limitation also holds for generators and comprehensions.
 
Together, these restrictions are quite strict. Some tips and observations:

 * Stick to very simple code generation primitives - like unrolling loops, etc.
 * Rely on inference and constant propagation as much as possible. For example, instead of using `promote_type` inside your function generator, emit it as generated code and let Julia calculate it as a part of the usual compilation process. This will avoid

In [ ]:
## EXERCISE

# Define a generated function that performs elementwise addition between two tuples

@generated function add_tuples(t1::NTuple{N, Any}, t2::NTuple{N, Any}) where N
    # Create expression
    
    # return expression
end

In [ ]:
# Test case

add_tuples((1, 2, 3), (2.2, 3.3, 4.4))

In [ ]:
## QUESTIONS?

# Closing remarks

I hope you've learnt something new (and useful!) during this tutorial. My goals have been to:

 * Get you to interact with tools like `@code_lowered` and `@code_typed` so you can explore Julia yourselves
 * Have a working understanding of how the Julia compiler works
 * Explore the fundamental metaprogramming capabilites provided by Julia
 * Highlight the limitations of each approach
 * Have a bit of fun playing around

I will finish with some general advice, from the perspective of a library writer:

 * As much as possible, try work with the natural compilation process - type inference, constant propagation and inlining.
 * Only intervene with macros, `@pure` functions and `@generated` functions when there is a compelling reason to do so.
 * Recursion works well for small sizes but is not suitable for large objects.
 * Use these powerful tools to make your API easier for the end user.
 * Try to maintain a minimal, generic and extensible interface. Blind composition with other libraries is key.
 * Macros are **not** extensible interfaces - they are one-trick ponies. (That can be OK!)
 * The metaprogramming magic should be invisible! (to users)
 * The metaprogramming magic should be readable... (to other developers)

In [ ]:
## QUESTIONS?